In [1]:
import pandas as pd
import os 
from scipy.interpolate import interp1d
from utils.data_plotting import *

### Load the texture data into a df

In [2]:
texture_df = pd.read_csv("texturas_extraidas_cast.csv")

### Create bins of 1cm (calling it TDEP)

In [3]:
# The interval we want
step = 0.01

# Floor division and round to the nearest below
texture_df['TDEP'] = (texture_df['Profundidade'] // step) * step

### Round all the texture variables to 4

In [4]:
texture_df = texture_df.round(4)

In [5]:
texture_df.to_csv('texturas_bins.csv', index=False)

### Use the mean and group by the bins

In [6]:
stats = {
    'contrast':      'mean',
    'dissimilarity': 'mean',
    'homogeneity':   'mean',
    'energy':        'mean',
    'correlation':   'mean',
    'entropy':       'mean'
}

results = texture_df.groupby('TDEP').agg(stats).reset_index()

### Round TDEP (e.g. 467.84000000000003 to 467.84)

In [7]:
results['TDEP'] = results['TDEP'].round(2)
results = results.round(4)

### Save the data into a CSV

In [8]:
results.to_csv('texturas_cast_res_preprocessed.csv', index=False)

### Get the convetional well logs from the same well

In [9]:
base_path = os.path.join('..', 'data', 'dlis_preprocessed_cast', '3-BRSA-778-SE', 'logical_file_0', 'frame_0.csv')
conventional_data = pd.read_csv(base_path)

### Testing with GR first

In [10]:
conventional_data = conventional_data.iloc[::-1].reset_index(drop=True)
gr_data = conventional_data[['TDEP', 'GR']]
gr_data = gr_data.dropna(subset=['GR'])

In [11]:
# 1. Crie o interpolador a partir dos seus dados originais de GR
interp_func = interp1d(
    gr_data['TDEP'],        # prof. originais do gr
    gr_data['GR'],          # valores de GR originais
    kind='linear',          # cria uma linha reta entre dois vals de GR.
)

# 2. Usando o intervalo de profundidade da imagem CAST
nova_tdep = results['TDEP'].values

min_gr_tdep = gr_data['TDEP'].min()
max_gr_tdep = gr_data['TDEP'].max()
nova_tdep = nova_tdep[(nova_tdep >= min_gr_tdep) & (nova_tdep <= max_gr_tdep)]


In [12]:
nova_tdep

array([427.12, 427.13, 427.14, ..., 689.88, 689.89, 689.9 ])

In [13]:
# 3. Gera valores de GR interpolados seguindo a reta entre dois pontos
nova_gr = interp_func(nova_tdep)

# 4. criando o df
gr_alinhado = pd.DataFrame({
    'TDEP': nova_tdep,
    'GR': nova_gr
})

print(gr_alinhado.head())

     TDEP        GR
0  427.12  97.07814
1  427.13  97.32286
2  427.14  97.56758
3  427.15  97.81230
4  427.16  98.05702


In [14]:
min_tdep = gr_alinhado['TDEP'].min()
max_tdep = gr_alinhado['TDEP'].max()

gr_data = gr_data[(gr_data['TDEP'] >= min_tdep) & (gr_data['TDEP'] <= max_tdep)]
print(gr_data.head())

       TDEP       GR
4117  427.2  99.0359
4118  427.3  93.0179
4119  427.4  91.1767
4120  427.5  91.8708
4121  427.6  95.5266


In [15]:
gr_data

,TDEP,GR
4117,427.2,99.0359
4118,427.3,93.0179
4119,427.4,91.1767
4120,427.5,91.8708
4121,427.6,95.5266
...,...,...
6740,689.5,125.0809
6741,689.6,134.6699
6742,689.7,135.7736
6743,689.8,133.0201


In [16]:
gr_alinhado

,TDEP,GR
0,427.12,97.07814
1,427.13,97.32286
2,427.14,97.56758
3,427.15,97.81230
4,427.16,98.05702
...,...,...
26274,689.86,127.41442
26275,689.87,126.48014
26276,689.88,125.54586
26277,689.89,124.61158


In [17]:
from matplotlib.backends.backend_pdf import PdfPages
# plot_gr_logs(gr_fino, "GR - High Resolution")
plot_gr_logs_2_wells_high_res(gr_data, gr_alinhado, "GR - Normal X High Resolution")
# Nome do arquivo PDF onde o gráfico será salvo
pdf_filename = "gr_normal_high_resolution_04.pdf"

# Criar o PDF e salvar o gráfico atual
with PdfPages(pdf_filename) as pdf:
    pdf.savefig()  # Salva o gráfico atual
    plt.close()    # Fecha o gráfico para evitar sobreposição

print(f"Gráfico salvo em {pdf_filename}")

Gráfico salvo em gr_normal_high_resolution_04.pdf
